In [1]:
from pathlib import Path
from multiprocessing import Pool
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import rebound as rb
import celmech as cm
import assist

In [2]:
from astroquery.jplsbdb import SBDB

In [5]:
SBDB.query("K08GG6Z", full_precision=True, covariance='mat', phys=True, get_uri=True)
SBDB._uri

'https://ssd-api.jpl.nasa.gov/sbdb.api?sstr=K08GG6Z&full-prec=true&cov=mat&phys-par=true'

In [2]:
ephem = assist.Ephem("../data/assist/linux_m13000p17000.441", "../data/assist/sb441-n16.bsp")

In [9]:
outer_only = True
print(f"Outer Planets Only: {outer_only}")
# %%
integration_path = Path("../data") / "uncertainty_integrations"
integration_path.mkdir(parents=True, exist_ok=True)
# %%
merged_df = pd.read_csv("../data/uncertainty_asteroids_sampled.csv", index_col=0)
start_time = 2460200.5

Outer Planets Only: True


In [10]:
if not outer_only:
    planets_list = [
    "Sun",
    "Mercury",
    "Venus",
    "Earth",
    "Mars",
    "Jupiter",
    "Saturn",
    "Uranus",
    "Neptune"
	]

planets_list = [
    "Jupiter",
    "Saturn",
    "Uranus",
    "Neptune"
]
    

In [11]:
def add_planets(sim):
	for name in planets_list:
		planet = ephem.get_particle(name, start_time - ephem.jd_ref)
		sim.add(planet)

In [16]:
def run_sim(r):
	idx, row = r

	# Integrate backwards to Nesvorny epoch
	sim = rb.Simulation()
	ex = assist.Extras(sim, ephem)
	# add_planets(sim)  
	# sim.move_to_hel()
	sim.add(x=row['x'], y=row['y'], z=row['z'], 
		 vx=row['vx']/365.25, vy=row['vy']/365.25, vz=row['vz']/365.25)
	
	# sim.add(x = 9.572786624350362E-01, y=-2.101947659454845E+00, z=-7.418444809938682E-01,
    #     vx = 9.715385994801106E-03, vy=6.153629531316274E-03, vz=1.549521077070705E-03)
	
	sim.t = row['epoch'] - ephem.jd_ref
	print(sim.t, start_time - ephem.jd_ref)
	# sim.integrate(start_time - ephem.jd_ref)
	sim.integrate(sim.t + 10_000)
	# sim.move_to_com()
	# sim.exit_max_distance = 50.

	# Create a new simulation and integrate forward
	ps_asteroid = sim.particles[0]
	ps_final = {
		'x': ps_asteroid.x,
		'y': ps_asteroid.y,
		'z': ps_asteroid.z,
		'vx': ps_asteroid.vx,
		'vy': ps_asteroid.vy,
		'vz': ps_asteroid.vz
	}
	print(row, sim.particles[0])

	sim2 = rb.Simulation()
	ex2 = assist.Extras(sim2, ephem)
	add_planets(sim2)
	sim2.move_to_hel()
	sim2.t = start_time - ephem.jd_ref

	# sim2.add(x=ps_final['x'], y=ps_final['y'], z=ps_final['z'], 
	# 	  vx=ps_final['vx'], vy=ps_final['vy'], vz=ps_final['vz'])
	
	# ps2 = sim2.particles
	# sim2.integrator = 'whfast'
	# sim2.dt = ps2[1].P/100.
	# sim2.ri_whfast.safe_mode = 0

	# Tfin_approx = 3e7*ps2[-1].P
	# total_steps = np.ceil(Tfin_approx / sim2.dt)
	# Tfin = total_steps * sim2.dt + sim2.dt
	# Nout = 25_000

	# sim_file = integration_path / f"asteroid_integration_{row["Des'n"]}-{idx}.sa"
	# sim2.save_to_file(str(sim_file), step=int(np.floor(total_steps/Nout)), delete_file=True)
	# sim2.integrate(Tfin, exact_finish_time=0)

run_sim(next(merged_df.iterrows()))

7366.5 8655.5
Des'n      K08GG6Z
epoch    2458911.5
x          1.21886
y        -2.516076
z         0.903672
vx        3.520961
vy        1.271503
vz        0.329221
type        random
Name: 0, dtype: object <rebound.particle.Particle object at 0x7f032f6fc550, m=0.0 x=0.7787567895267897 y=-2.667956581281581 z=0.8548091287959108 vx=0.010130985129163566 vy=0.002045189918175092 vz=0.0013546291611699983>


In [15]:
# with Pool(40) as p:
#       p.map(run_sim, merged_df.iterrows())